In [6]:
import tkinter as tk
import tkinter.filedialog as filedialog
import tkinter.messagebox as messagebox
import requests
import base64
import io
import os
from PIL import Image, ImageFilter, ImageTk
from tkinter import ttk


In [3]:
# Get the path to the desktop
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

url = "https://api.stability.ai/v1/generation/stable-diffusion-xl-1024-v1-0/text-to-image"

body = {
  "steps": 40,
  "width": 1024,
  "height": 1024,
  "seed": 0,
  "cfg_scale": 5,
  "samples": 1,
  "style_preset": "anime",
  "text_prompts": [
    {
      "text": "devil on earth",
      "weight": 1
    },
    {
      "text": "anime,evil",
      "weight": -1
    }
  ],
}

api_key = "sk-Q3RkYKmaSHBeH8OTLDcxKyzPn4jOhZkwFpDjNzOqbstQvRrb"

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": api_key,
}

response = requests.post(
  url,
  headers=headers,
  json=body,
)

if response.status_code != 200:
    raise Exception("Non-200 response: " + str(response.text))

data = response.json()

# make sure the out directory exists on the desktop
out_path = os.path.join(desktop_path, "out")
if not os.path.exists(out_path):
    os.makedirs(out_path)

quality = 5

#new_width, new_height = 512, 512  # New dimensions for downscaling

blur_radius = 1

for i, image in enumerate(data["artifacts"]):
    image_path = os.path.join(out_path, f'txt2img_{image["seed"]}.png')
    with open(image_path, "wb") as f:
        f.write(base64.b64decode(image["base64"]))
    
    # Open the image file
    img = Image.open(image_path)
    # Apply a blur filter
    img_blurred = img.filter(ImageFilter.GaussianBlur(blur_radius))
    # Save the image with reduced quality
    img_blurred.save(image_path, 'JPEG', quality=quality)
    
    # Display the image using a shell command
    subprocess.run(["start", image_path], shell=True)


In [8]:
#GUI
class ImageGenerationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Generation GUI")
        self.root.geometry("1250x1550")  # Fixed window size

        # Variable to store user input
        self.text_prompt_1 = ""
        self.text_prompt_2 = ""
        self.image_size_options = ["1024x1024", "1152x896", "1216x832"]
        self.selected_image_size = tk.StringVar(value=self.image_size_options[0])

        # Create GUI elements
        self.create_input_widgets()
        self.create_buttons()
        self.create_image_label()
        self.create_processing_label()

    def create_input_widgets(self):
        input_frame = ttk.Frame(self.root)
        input_frame.pack(padx=10, pady=10, fill='x')

        ttk.Label(input_frame, text="Enter What You Wanna See:").grid(row=0, column=0, sticky='w')
        self.text_entry_1 = ttk.Entry(input_frame, width=40)
        self.text_entry_1.insert(0, self.text_prompt_1)
        self.text_entry_1.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(input_frame, text="Add Features:").grid(row=1, column=0, sticky='w')
        self.text_entry_2 = ttk.Entry(input_frame, width=40)
        self.text_entry_2.insert(0, self.text_prompt_2)
        self.text_entry_2.grid(row=1, column=1, padx=5, pady=5)

        ttk.Label(input_frame, text="Image Size:").grid(row=2, column=0, sticky='w')
        size_dropdown = ttk.Combobox(input_frame, textvariable=self.selected_image_size, values=self.image_size_options)
        size_dropdown.grid(row=2, column=1, padx=5, pady=5)

    def create_buttons(self):
        button_frame = ttk.Frame(self.root)
        button_frame.pack(padx=10, pady=10)

        ttk.Button(button_frame, text="Generate", command=self.display_processing_animation).grid(row=0, column=0, padx=5, pady=5)
        ttk.Button(button_frame, text="Download", command=self.download_image).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(button_frame, text="Close", command=self.close_app).grid(row=0, column=2, padx=5, pady=5)

    def create_image_label(self):
        self.image_label = ttk.Label(self.root)
        self.image_label.pack(pady=10)

    def create_processing_label(self):
        self.processing_label = ttk.Label(self.root, text="Generating image...")
        self.processing_label.pack_forget()

    def display_processing_animation(self):
        # Hide the image label and show the processing label
        self.image_label.pack_forget()
        self.processing_label.pack()

        # Schedule the image loading process after a short delay
        self.root.after(100, self.generate_image)

    def generate_image(self):
        # Actual image loading process
        self.text_prompt_1 = self.text_entry_1.get()
        self.text_prompt_2 = self.text_entry_2.get()

        selected_size = self.selected_image_size.get().split("x")
        width, height = int(selected_size[0]), int(selected_size[1])

        uxx = "https://api.stability.ai/v1/generation/stable-diffusion-xl-1024-v1-0/text-to-image"
        body = {
            "steps": 40,
            "width": width,
            "height": height,
            "seed": 0,
            "cfg_scale": 5,
            "samples": 1,
            "style_preset": "anime",
            "text_prompts": [
                {"text": self.text_prompt_1, "weight": 1},
                {"text": self.text_prompt_2, "weight": -1}
            ],
        }

        
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization":"sk-Q3RkYKmaSHBeH8OTLDcxKyzPn4jOhZkwFpDjNzOqbstQvRrb",
        }

        response = requests.post(uxx, headers=headers, json=body)

        if response.status_code == 200:
            data = response.json()
            artifacts = data.get("artifacts", [])
            if artifacts:
                image_data = base64.b64decode(artifacts[0]["base64"])
                img = Image.open(io.BytesIO(image_data))

                
                img_blurred = img.filter(ImageFilter.GaussianBlur(2))

                # Save the image to a file using PIL
                output_image_path = "generated_image.jpg"
                img_blurred.save(output_image_path, 'JPEG', quality=6*2)

                # Display the image in the Tkinter label
                img = Image.open(output_image_path)
                img = ImageTk.PhotoImage(img)
                self.image_label.config(image=img)
                self.image_label.image = img

                # Hide the processing label and show the image label
                self.processing_label.pack_forget()
                self.image_label.pack()
            else:
                messagebox.showerror("Error", "No image artifacts found in the response.")
        else:
            error_message = f"Image generation failed: {response.text}"
            messagebox.showerror("Error", error_message)


    def download_image(self):
        try:
            if os.path.exists("generated_image.jpg"):
                filename = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg")])
                if filename:
                    os.rename("generated_image.jpg", filename)
            else:
                messagebox.showerror("Error", "Generated image not found.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def close_app(self):
        self.root.destroy()


if __name__ == "__main__":
    root = tk.Tk()
    app = ImageGenerationApp(root)
    root.mainloop()
